# 02-2 스크래핑 사용하기

In [220]:
from bs4 import BeautifulSoup

## 전체 도서의 무게 구하기

In [221]:
def get_gram_cnt(isbn):
    # Yes24 도서 검색 페이지 URL
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    # URL에 ISBN을 넣어 HTML 가져옵니다.
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    # 검색 결과에서 해당 도서를 선택합니다.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 도서 상세 페이지를 가져옵니다.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    # 상품 상세정보 div를 선택합니다.
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    # 테이블에 있는 tr 태그를 가져옵니다.
    prd_tr_list = prd_detail.find_all('tr')
    # 쪽수가 들어 있는 th를 찾아 td에 담긴 값을 반환합니다.
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[2]
    return ''

In [222]:
get_gram_cnt(9791190090018)

'496g'

In [223]:
top10_books = books.head(10)

In [224]:
def get_gram_cnt2(row):
    isbn = row['isbn13']
    return get_gram_cnt(isbn)

In [225]:
gram_count = top10_books.apply(get_gram_cnt2, axis=1)
print(gram_count)

0    496g
1    358g
2    296g
3    412g
4    388g
5    512g
6    318g
7        
8    324g
9    406g
dtype: object


In [226]:
gram_count = top10_books.apply(lambda row: get_gram_cnt(row['isbn13']), axis=1)

In [227]:
gram_count.name = 'gram_count'
print(gram_count)

0    496g
1    358g
2    296g
3    412g
4    388g
5    512g
6    318g
7        
8    324g
9    406g
Name: gram_count, dtype: object


In [228]:
top10_with_gram_count = pd.merge(top10_books, gram_count, 
                                 left_index=True, right_index=True)
top10_with_gram_count

,no,ranking,bookname,authors,publisher,publication_year,isbn13,gram_count
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,496g
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,358g
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,296g
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,412g
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,388g
5,6,6,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,512g
6,7,7,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,318g
7,8,8,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,
8,9,9,선량한 차별주의자,김지혜 지음,창비,2019,9788936477196,324g
9,10,9,쇼코의 미소 :최은영 소설,지은이: 최은영,문학동네,2016,9788954641630,406g


### 옵션

In [229]:
def get_page_cnt(isbn):
    url = 'http://www.yes24.com/Product/Search?domain=BOOK&query={}'
    r = requests.get(url.format(isbn))
    soup = BeautifulSoup(r.text, 'html.parser')   # HTML 파싱
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    prd_detail = soup.find('div', attrs={'id':'infoset_specific'})
    prd_tr_list = prd_detail.find_all('tr')
    for tr in prd_tr_list:
        if tr.find('th').get_text() == '쪽수, 무게, 크기':
            return tr.find('td').get_text().split()[0]
    return ''

In [230]:
def get_page_cnt2(row):
    isbn = row['isbn13']
    return get_page_cnt(isbn)

In [231]:
page_count = top10_books.apply(get_page_cnt2, axis=1)
print(page_count)

0    330쪽
1    300쪽
2    228쪽
3    340쪽
4    264쪽
5    396쪽
6    272쪽
7        
8    244쪽
9    296쪽
dtype: object


In [232]:
page_count = top10_books.apply(lambda row: get_page_cnt(row['isbn13']), axis=1)

In [233]:
page_count.name = 'page_count'
print(page_count)

0    330쪽
1    300쪽
2    228쪽
3    340쪽
4    264쪽
5    396쪽
6    272쪽
7        
8    244쪽
9    296쪽
Name: page_count, dtype: object


In [235]:
top10_with_etc_count = pd.merge(top10_with_gram_count, page_count,
                                 left_index=True, right_index=True)
top10_with_etc_count

,no,ranking,bookname,authors,publisher,publication_year,isbn13,gram_count,page_count
0,1,1,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,496g,330쪽
1,2,2,달러구트 꿈 백화점.이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,358g,300쪽
2,3,3,지구에서 한아뿐 :정세랑 장편소설,지은이: 정세랑,난다,2019,9791188862290,296g,228쪽
3,4,4,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,412g,340쪽
4,5,5,아몬드 :손원평 장편소설,지은이: 손원평,창비,2017,9788936434267,388g,264쪽
5,6,6,피프티 피플 :정세랑 장편소설,지은이: 정세랑,창비,2016,9788936434243,512g,396쪽
6,7,7,목소리를 드릴게요 :정세랑 소설집,지은이: 정세랑,아작,2020,9791165300005,318g,272쪽
7,8,8,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,9788972756194,,
8,9,9,선량한 차별주의자,김지혜 지음,창비,2019,9788936477196,324g,244쪽
9,10,9,쇼코의 미소 :최은영 소설,지은이: 최은영,문학동네,2016,9788954641630,406g,296쪽
